In [6]:
import pandas as pd
import numpy as np
import pickle
import openai
import time
import os
import re

from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [7]:
np.random.seed(69)

In [22]:
with open("./df_pickle/df_context.pkl", "rb") as fp:
    df_context = pickle.load(fp)

In [23]:
with open("./df_pickle/df_questions.pkl", "rb") as fp:
    pool_question = pickle.load(fp)

In [10]:
test_questions = pool_question.sample(n=6)
pool_question.drop(test_questions.index, inplace=True)
gen_questions = pool_question.sample(n=6)
pool_question.drop(gen_questions.index, inplace=True)

pool_question.reset_index(drop=True, inplace=True)
test_questions.reset_index(drop=True, inplace=True)
gen_questions.reset_index(drop=True, inplace=True)

In [6]:
print('Mean context length (chars):', df_context['context'].apply(lambda x: len(x)).values.mean())

Mean context length (chars): 3791.13125


In [7]:
fred_ct = 'A Bayesian network also called a belief network or Bayesian belief network is not as such adding a new method to our toolbox, but it provides a convenient and often-used notation for presenting existing probabilities. Consider the following example adapted from Pearl [2014], MacKay [2003] Fred lives in Los Angeles and commutes 60 miles to work. Whilst at work, he receives a phone-call from his neighbour saying that Fred’s burglar alarm is ringing. What is the probability that there was a burglar in his house today? While driving home to investigate, Fred hears on the radio that there was a small earthquake that day near his home. ‘Oh’, he says, feeling relieved, ‘it was probably the earthquake that set off the alarm’. What is the probability that there was a burglar in his house? To analyse this story we first introduce the variables: a : The alarm is ringing. b : A burglar was in Fred’s house. c : Fred received a phone-call reporting the alarm. e : A small earthquake took place today near Fred’s house. r : The radio report of the earthquake is heard by Fred. In a case like this, we know (from our experience) that some of these events must be independent. That there is a burglar or a minor earthquake is presumable unrelated events, so p(b, e) = p(b)p(e). In general, the probability of these variables will factorize as follows:232 13 Bayesian methods Earthquake Burglar Radio Alarm Phonecall Fig. 13.3. Bayesian network of the burglar example. Each vertex corresponds to a variable, and incident edges corresponds to conditional dependence. p(a, b, c, e, r) = p(b)p(e)p(a|b, e)p(c|a)p(r|e). (13.14) This factorization of the probability has important consequences. Firstly, as for the na¨ıve-Bayes assumption, it makes the probability density much less costly to store on a computer and reliable to estimate as there are fewer parameters than the full (un-factorized) joint distribution. Secondly, it allows faster computation by exploiting the factorization structure and finally it allows us easier to see what quantities are independent of each other. It is common to represent the factorization as a network where the vertices correspond to the variables and the edges correspond to statistical dependence, see fig. 13.3 for an illustration. So for instance, if there is an edge from B to A, that means that in the joint distribution, then A must be conditional on B and possible other variables connected to A. To solve the Burglar problem, assume the probability of there being a burglar is p(b = 1) = 0.1%, earthquake p(e = 1) = 0.1% (corresponding to roughly one burglar and earthquake every four years) and that the alarm is triggered either by (1) false alarms (very low probability), (2) if an earthquake takes place (low probability) and finally if a burglar enters the home (high probability). In our example these probabilities are:1 1 For instance, suppose we let f = 0.1% denote the chance a false alarm triggers a, αe = 1% the chance an earthquake triggers a and finally αb = 99% the chance a burglar triggers a. The probabilities can then be obtained as: p(a = 1|b = 0, e = 0) = f, p(a = 1|b = 0, e = 1) = 1−(1−f)(1−αe), p(a = 1|b = 1, e = 0) = 1−(1−f)(1−αb), p(a = 1|b = 1, e = 1) = 1−(1−f)(1−αb)(1−αe).13.3 Bayesian networksF 233 p(a = 1|b = 0, e = 0) = 0.1%, p(a = 1|b = 0, e = 1) = 1.099%, p(a = 1|b = 1, e = 0) = 99.001%, p(a = 1|b = 1, e = 1) = 99.011%. Finally assume the neighbour would never phone if the alarm is not ringing (p(c = 1|a = 0) = 0) and that the radio reported is also trustworthy (p(r = 1|e = 0) = 0) and let’s return to the problem: Suppose first the phone calls c = 1; then we know the alarm is ringing a = 1 and so the posterior probability of b, e (burglary and earthquake) becomes: p(b, e|a = 1) = p(a = 1|b, e)p(b, e) p(a = 1) . We can use the Bayes network to compute these probabilities. For instance when computing p(a = 1), we must compute this by summing over all other variables than a: p(a = 1) = X b∈{0,1} X c∈{0,1} X e∈{0,1} X r∈{0,1} p(a = 1, b, c, e, r), (13.15) However, if we plug in the expression of the likelihood (13.14) we see that variables c and r can trivially be summed out (i.e., marginalized): p(a = 1) = X b∈{0,1} X c∈{0,1} X e∈{0,1} X r∈{0,1} p(b)p(e)p(a = 1|b, e)p(c|a = 1)p(r|e) = X b∈{0,1} X e∈{0,1} \uf8ee \uf8f0p(b)p(e)p(a = 1|b, e) \uf8eb \uf8ed X c∈{0,1} p(c|a = 1) X r∈{0,1} p(r|e) \uf8f6 \uf8f8 \uf8f9 \uf8fb = X b∈{0,1} X e∈{0,1} p(b)p(e)p(a = 1|b, e) (13.16) Comparing to the Bayesian network in fig. 13.3 we see that to determine what variables remain in the sum when computing p(a), we look at all other vertices in a network where we can move to a by going in the direction of the edges. See fig. 13.4 where we have illustrated the two nodes that remain, e, b, with red. Using the above numbers we obtain: p(a = 1|b = 0, e = 0)p(b = 0)p(e = 0) = 0.000998, p(a = 1|b = 1, e = 0)p(b = 1)p(e = 0) = 0.0000989, p(a = 1|b = 0, e = 1)p(b = 0)p(e = 1) = 0.000010979, p(a = 1|b = 1, e = 1)p(b = 1)p(e = 1) = 9.9 × 10−7 . By inserting these four numbers into eq. (13.16) and summing we obtain p(a = 1) = 0.002 and so from eq. (13.15) p(b = 0, e = 0|a = 1) = 0.4993, (13.17) p(b = 1, e = 0|a = 1) = 0.4947, (13.18) p(b = 0, e = 1|a = 1) = 0.0055, (13.19) p(b = 1, e = 1|a = 1) = 0.0005. (13.20)234 13 Bayesian methods Earthquake Burglar Radio Alarm Phonecall Fig. 13.4. To determine what variables must be summed out when computing the marginal of a variable such as a, we look at all variables such that one can move in the direction of the arrows from those variables to a. This gives p(a, e, b) So returning to the initial question, when we determine if there was a burglar at the house we must compute p(b = 1|a = 1) which can be accomplished by marginalizing over the burglar-variable: p(b = 0|a = 1) = p(b = 0, e = 0|a = 1) + p(b = 0, e = 1|a = 1) = 0.505 p(b = 1|a = 1) = p(b = 1, e = 0|a = 1) + p(b = 1, e = 1|a = 1) = 0.494 so after receiving the call, we believe there to be a 50% chance there was a burglar in the house. An important point to take away from this example is that b and e, which were initially independent: p(e, b) = p(e)p(b), are made dependent by the information a. Now consider the final part of the example. Suppose we also learn that e = 1 (i.e. there was an earthquake). The probability there was a burglar can now be computed as: p(b|e, a) = p(b, e|a) p(e|a) = p(b, e|a) p(e, b = 0|a) + p(e, b = 1|a) . If we plug in numbers we obtain p(e = 1|a = 1) = 0.006 and so p(b = 0|e = 1, a = 1) = p(b = 0, e = 1|a = 1) p(e = 1|a = 1) = 0.92 p(b = 1|e = 1, a = 1) = p(b = 1, e = 1|a = 1) p(e = 1|a = 1) = 0.08 So after learning the alarm was triggered, this lowers our probability there was a burglar in the house from about 50% to about 8%. This is in according to everyday intuition: when we learn about the earthquake, we consider that to be the more plausible explanation of the alarm.'

# Oracle - Getting label from ChatGPT

In [8]:
max_req_per_min = 19

In [45]:
test_data = []
req_per_min = 0
last_time_stamp = time.time()
#For each pair of questions
for qi in range(0,len(test_questions),2):
    q1 = test_questions.iloc[qi]
    q2 = test_questions.iloc[qi+1]
    
    #For each context
    for ci,ctx in enumerate(df_context['context']):    
        #Check rate limit
        req_per_min += 1
        while req_per_min>=max_req_per_min:
            time_stamp = time.time()
            if int(time.time()-last_time_stamp) > 60:
                last_time_stamp = time_stamp
                req_per_min = 0
            else:
                time.sleep(10)
        
        #Ask the oracle for label for the context and two questions
        try:
            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role":"system", "content":"You are a system designed to label if a list of provided questions can be answered using ANY part of a provided context. You will always only reply in the following format for each question: `label: LABEL. LABEL should be 'y' if a question can be answered using the context and else 'n'"},
                {"role":"user", "content": "CONTEXT: ```The man in the house has a boy named Bob and a red car. He loves ice cream``` QUESTION: ```Is the boy named Jim?``` QUESTION: ```Does the man have a red book?```"},
                {"role":"assistant", "content":"label: y\nlabel: n"},
                {"role": "user", "content": f'CONTEXT: ```{ctx}``` QUESTION: ```{q1["question"]}``` QUESTION: ```{q2["question"]}```'}
            ]
            )
        except Exception as e:
            print("OPENAI_ERROR:",str(e))
            continue

        
        
        #Parse the response to get labels
        res = completion.choices[0].message.content
        labels = re.findall(r'label: ([yn])', res)
        
        #If two labels are not found, assume something went wrong and skip this iteration
        if len(labels) != 2:
            print('[Error] labels not found in response:', res)
            continue
        
        #Add the two questions, context and labels to the training data
        test_data.append({'question': q1["question"], 'context': ctx, 'question_embedding': q1["embedding"], 'label': labels[0]})
        test_data.append({'question': q2["question"], 'context': ctx, 'question_embedding': q2["embedding"], 'label': labels[1]})
        
        #If count is divisible by 100, save a backup
        cnt = qi*len(df_context)+ci
        if cnt % 100 == 0:
            print(cnt)
            with open(f'./oracle_bu/test/{cnt}.pkl', 'wb') as fp:
                pickle.dump(test_data, fp)

0
100


KeyboardInterrupt: 

In [ ]:
df_test = pd.DataFrame(data=test_data, columns=['question','context','question_embedding','context_embedding','label'])

In [9]:
gen_data = []
req_per_min = 0
last_time_stamp = time.time()
#For each pair of questions
for qi in range(0,len(gen_questions),2):
    q1 = gen_questions.iloc[qi]
    q2 = gen_questions.iloc[qi+1]
    
    #For each context
    for ci,ctx in enumerate(df_context['context']):
        #Check rate limit
        req_per_min += 1
        while req_per_min>=max_req_per_min:
            time_stamp = time.time()
            if int(time.time()-last_time_stamp) > 60:
                last_time_stamp = time_stamp
                req_per_min = 0
            else:
                time.sleep(10)
        
        #Ask the oracle for label for the context and two questions
        try:
            completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role":"system", "content":"You are a system designed to label if a list of provided questions can be answered using ANY part of a provided context. You will always only reply in the following format for each question: `label: LABEL. LABEL should be 'y' if a question can be answered using the context and else 'n'"},
                {"role":"user", "content": "CONTEXT: ```The man in the house has a boy named Bob and a red car. He loves ice cream``` QUESTION: ```Is the boy named Jim?``` QUESTION: ```Does the man have a red book?```"},
                {"role":"assistant", "content":"label: y\nlabel: n"},
                {"role": "user", "content": f'CONTEXT: ```{ctx}``` QUESTION: ```{q1["question"]}``` QUESTION: ```{q2["question"]}```'}
            ]
            )
        except Exception as e:
            print("OPENAI_ERROR:",str(e))
            continue
        
        #Parse the response to get labels
        res = completion.choices[0].message.content
        labels = re.findall(r'label: ([yn])', res)
        
        #If two labels are not found, assume something went wrong and skip this iteration
        if len(labels) != 2:
            print('[Error] labels not found in response:', res)
            continue
        
        #Add the two questions, context and labels to the training data
        gen_data.append({'question': q1["question"], 'context': ctx, 'question_embedding': q1["embedding"], 'label': labels[0]})
        gen_data.append({'question': q2["question"], 'context': ctx, 'question_embedding': q2["embedding"], 'label': labels[1]})
        
        #If count is divisible by 100, save a backup
        cnt = qi*len(df_context)+ci
        if cnt % 100 == 0:
            print(cnt)
            with open(f'./oracle_bu/gen/{cnt}.pkl', 'wb') as fp:
                pickle.dump(gen_data, fp)

0
100
400
OPENAI_ERROR: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)
700


In [10]:
df_gen = pd.DataFrame(data=gen_data, columns=['question','context','question_embedding','context_embedding','label'])

In [ ]:
with open(f'./oracle_bu/gen/df_gen.pkl', 'wb') as fp:
                pickle.dump(df_gen, fp)

In [14]:
df_gen['label'].value_counts()

n    571
y    387
Name: label, dtype: int64

In [19]:
df_gen[df_gen['label'] == 'y']['question'].value_counts()

What is cross-validation and how does it help in selecting an appropriate model?                 109
What is a dataset?                                                                                94
What is the basic structure of an ANNs and how is it similar to biological neurons?               72
What are some common data issues encountered in datasets?                                         66
What is the concept of entropy in information theory?                                             32
What are the three attributes observed for each patient in the dataset mentioned in the text?     14
Name: question, dtype: int64

In [11]:
#shuffled list of dicts of two question indicies and context indices
cq_idx = []
for ci in df_context.index:
     for q1,q2 in zip(pool_question.index[::2], pool_question.index[1::2]):
        cq_idx.append({'q1i':q1, 'q2i':q2, 'ci':ci})
np.random.shuffle(cq_idx)

In [21]:
len(cq_idx)

14240

In [20]:
df_context.iloc[[idx['ci'] for idx in cq_idx[4000:]]]['context']

5       We might not notice, but machine learning is ...
125     The above presentation leaves two important q...
10      Let’s make the above discussion more concrete...
52      In this section, we will consider two example...
36      We have purposefully introduced probabilities...
                             ...                        
37      Data will often come in numerical form, in wh...
157    10.1.4 Cross-validation for model selection We...
37      Data will often come in numerical form, in wh...
56      Consider once again our rain-fall example fro...
56      Consider once again our rain-fall example fro...
Name: context, Length: 10240, dtype: object

In [27]:
train_data = []
n_train = 2000

req_per_min = 0
last_time_stamp = time.time()

#For num train points
for i, idx in enumerate(cq_idx[:n_train]):
    ctx = df_context.iloc[idx['ci']]['context']
    q1 = pool_question.iloc[idx['q1i']]
    q2 = pool_question.iloc[idx['q2i']]

    #Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(10)
    
    #Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
            {"role":"system", "content":"You are a system designed to label if a list of provided questions can be answered using ANY part of a provided context. You will always only reply in the following format for each question: `label: LABEL. LABEL should be 'y' if a question can be answered using the context and else 'n'"},
            {"role":"user", "content": "CONTEXT: ```The man in the house has a boy named Bob and a red car. He loves ice cream``` QUESTION: ```Is the boy named Jim?``` QUESTION: ```Does the man have a red book?```"},
            {"role":"assistant", "content":"label: y\nlabel: n"},
            {"role": "user", "content": f'CONTEXT: ```{ctx}``` QUESTION: ```{q1["question"]}``` QUESTION: ```{q2["question"]}```'}
        ]
        )
    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue
    
    #Parse the response to get labels
    res = completion.choices[0].message.content
    labels = re.findall(r'label: ([yn])', res)
    
    #If two labels are not found, assume something went wrong and skip this iteration
    if len(labels) != 2:
        print('[Error] labels not found in response:', res)
        continue
    
    #Add the two questions, context and labels to the training data
    train_data.append({'question': q1["question"], 'context': ctx, 'question_embedding': q1["embedding"], 'label': labels[0]})
    train_data.append({'question': q2["question"], 'context': ctx, 'question_embedding': q2["embedding"], 'label': labels[1]})
    
    #If iteration num is divisible by 100, save a backup
    if i % 100 == 0:
        print(i)
        with open(f'./oracle_bu/train/{i}.pkl', 'wb') as fp:
            pickle.dump(train_data, fp)

0
[Error] labels not found in response: label: y
label: The context does not provide a clear answer to this question.
[Error] labels not found in response: label: y
label: 5


KeyboardInterrupt: 

In [26]:
df_train = pd.DataFrame(data=train_data, columns=['question','context','question_embedding','context_embedding','label'])